<a href="https://colab.research.google.com/github/6760525/6760525/blob/main/seabattle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports and Constants

In [1]:
#Imports
from random import randint

#Constants

# The game's square field size
FIELD_SIZE = 6 
#FIELD_SIZE = 10

# The game's fleet structure: key - number of decks, value - number of ships
FLEET = {3: 1, 2: 2, 1: 4} 
#FLEET = {4:1, 3: 2, 2: 3, 1: 4}

EMPTY_CHAR = chr(0x25CC)
SHIP_CHAR = chr(0x25A3)
CONTOUR_CHAR = chr(0x25AB)




# Internal logic of the game

## Classes of Exceptions:

In [2]:
# Classes of Exceptions:
class BoardOutException(BaseException):
  pass

class SpotBusyException(BaseException):
  pass

class OutOfSpaceException(BaseException):
  pass


## Dots and Ships


### Dot is a class for coordinates on the game's field

In [3]:
# Dot is a class for coordinates on the game's field
class Dot:
  def __init__(self, x, y):
    self.x = x
    self.y = y

  def get_x(self):
    return self.x

  def get_y(self):
    return self.y

  # checks if a dot is on the board
  def __eq__(self, other):
    return self.x == other.x and self.y == other.y

  def __str__(self):
    return f'Dot: {self.x, self.y}'


### Ship is a class for a ship

In [4]:
# Ships
class Ship:
  def __init__(self, length, dot, dir, lifes):
    self.length = length
    self.dot = dot
    self.dir = dir
    self.lifes = lifes

  def dots(self):
    # Returns list of all ship's dots
    dots_list = []
    x = self.dot.get_x()
    y = self.dot.get_y()
    dx, dy = (0, 1) if self.dir == 1 else (1, 0)
    for i in range(self.length):
      dots_list.append(Dot(x + i * dx, y + i * dy))
    return(dots_list)



## Board class


In [5]:
class Board:
  def __init__(self):
    self.field = [[EMPTY_CHAR] * FIELD_SIZE for _ in range(FIELD_SIZE) ]
    self.adots = []
    self.ships = []
    self.hid = True
    self.lifes = 7
    for i in range(FIELD_SIZE):
      for j in range(FIELD_SIZE):
        self.adots.append(Dot(i,j))

  def add_ship(self, ship):
    if self.adots == []:
      raise OutOfSpaceException

    sd = ship.dots()
    if all(d in self.adots for d in sd):
      self.adots = [d for d in self.adots if (d not in sd)]
      for d in sd: self.field[d.x][d.y] = SHIP_CHAR
      self.ships.append(ship)
      self.contour(ship)
    else:
      raise BoardOutException

  def contour(self, ship):
    for d in ship.dots():
      for i in (-1, 0, 1):
        for j in (-1, 0, 1):
          cx = d.get_x() + i
          cy = d.get_y() + j
          cd = Dot(cx, cy)
          if cd in self.adots:
            self.adots.remove(cd)
            if self.field[cx][cy] == EMPTY_CHAR:
              self.field[cx][cy] = CONTOUR_CHAR 

  def show_board(self, hid=False):
    print('-' * (4 * FIELD_SIZE + 4))
    print('  ', end='')
    for i in range(1, len(self.field[0])+1): print(f' |{i:2}', end='')
    print(' |')
    print('-' * (4 * FIELD_SIZE + 4))

    if hid:
      pass
    else:
      for i in range(len(self.field)):
        print(f'{i+1:2} | ', end='')
        for j in range(len(self.field[i])):
          print(f'{self.field[i][j]} | ', end='')
        print()
    print('-' * (4 * FIELD_SIZE + 4))

  def out(self):
    pass

  def shot(self, dot):
    if dot in Board:
        pass


# Player class and sub-classes

In [46]:
class Player:
  def __init__(self, board, oppboard):
    self.board = board
    self.oppboard = oppboard

  def ask(self):
    pass

  def move(self):
    self.ask()
    self.oppboard.move()

class AI(Player):
  def ask(self):
    dot = Dot()
    if 1 != 1: # if damaged ship(s) exist(s), continue to shoot at it(them)
      pass
    else:
      # выбрать случайную свободную клетку
      dot = self.oppboard.adots[randint(0, len(self.oppboard.adots)-1)]
      print(dot)
                                  
                                        
    return(dot)

class User(Player):
  def ask(self):
    x, y = (int(x) for x in input().split())
    dot = Dot(x, y)
    if dot in self.oppboard.adots:
      return(dot)
    else:
      raise BoardOutException


# The external logic of the game


## User Interface

### Game class


In [50]:
class Game:
  def __init__(self):
    self.user_board = Board()
    self.user_oppboard = Board()
    self.ai_board = Board()
    self.ai_oppboard = Board()

    self.user = User(self.user_board, self.user_oppboard)
    self.ai = AI(self.ai_board, self.ai_oppboard)
     
  def random_board(self, board):
    print('Now AI is placing ships on the board...')
    for sub_fleet in list(FLEET.items()):
      for i in range(1, sub_fleet[1]+1):
        while True:
          try:
            if board.adots == []:
              raise OutOfSpaceException
            dot = board.adots[randint(0, len(board.adots)-1)]
            vh = randint(0, 1)
            row = dot.get_x()
            col = dot.get_y()
            board.add_ship(Ship(sub_fleet[0], dot, vh, sub_fleet[0]))
            break
          except SpotBusyException:
            print('This cell is busy! Try again.')
          except BoardOutException:
            print('This cell is out of field! Try again.')
          except ValueError:
            print('It''s required two numbers at least! Try again.')
          except OutOfSpaceException:
            print('No free spots on the field! Try again.')
            return(-2)
    return(0)      


  def greet(self):
    print('Hello! Here is your field. Please put your ships on it:')
    self.user_board.show_board()

    for sub_fleet in list(FLEET.items()):
      for i in range(1, sub_fleet[1]+1):
        while True:
          try:
            if self.user_board.adots == []:
              raise OutOfSpaceException

            if sub_fleet[0] == 1:
              ship_data = input(f'{sub_fleet[0]}-deck ship #{i}: row, col: ').split()
            else:
              ship_data = input(f'{sub_fleet[0]}-deck ship #{i}: row, col, direction (0=v, 1=h): ').split()

            if ship_data[0] == ':q':
              return (-1)

            if len(ship_data) < 2:
              raise ValueError
            elif len(ship_data) == 2:
              ship_data.append('0')

            row = int(ship_data[0])
            col = int(ship_data[1])
            dir = int(ship_data[2])
            self.user_board.add_ship(Ship(sub_fleet[0], Dot(row-1, col-1), 0 if dir == 0 else 1, sub_fleet[0]))
            self.user_board.show_board()
            break
          except SpotBusyException:
            print('This cell is busy! Try again.')
          except BoardOutException:
            print('This cell is out of field! Try again.')
          except ValueError:
            print('It''s required two numbers at least! Try again.')
          except OutOfSpaceException:
            print('No free spots on the field! Try again.')
            return(-2)
    return(0)      

  def loop(self):
    if input('Do you want to start? [y/N]').upper() == 'Y':
      self.user.move()
    else:
      self.ai.move()


  def start(self):
    if input('Do you need a help with settele your fleet? [y/N]').upper() == 'Y':
      self.random_board(self.user_board)
      print('This is your board. AI has just placed your ships on your board on your behalf:')
      self.user_board.show_board()
    else:
      self.greet()

    self.random_board(self.ai_board)
    print('This is AI board. For test purposes only!')
    self.ai_board.show_board()

    self.loop()

# Artificial Intelligence

## Game Controller (counts a number of wrecked ships)

In [51]:
# Artificial Intelligence


# Main

In [52]:
# Main
if __name__ == '__main__':
  game = Game()
  game.start()


Do you need a help with settele your fleet? [y/N]y
Now AI is placing ships on the board...
This cell is out of field! Try again.
This is your board. AI has just placed your ships on your board on your behalf:
----------------------------
   | 1 | 2 | 3 | 4 | 5 | 6 |
----------------------------
 1 | ▫ | ▫ | ▫ | ▫ | ▫ | ▫ | 
 2 | ▫ | ▣ | ▣ | ▣ | ▫ | ▣ | 
 3 | ▫ | ▫ | ▫ | ▫ | ▫ | ▫ | 
 4 | ▫ | ▣ | ▫ | ▣ | ▫ | ▣ | 
 5 | ▫ | ▫ | ▫ | ▫ | ▫ | ▣ | 
 6 | ▣ | ▫ | ▣ | ▣ | ▫ | ▫ | 
----------------------------
Now AI is placing ships on the board...
This cell is out of field! Try again.
This cell is out of field! Try again.
This cell is out of field! Try again.
This cell is out of field! Try again.
This cell is out of field! Try again.
No free spots on the field! Try again.
This is AI board. For test purposes only!
----------------------------
   | 1 | 2 | 3 | 4 | 5 | 6 |
----------------------------
 1 | ▣ | ▫ | ▫ | ▫ | ▫ | ▫ | 
 2 | ▣ | ▫ | ▣ | ▣ | ▣ | ▫ | 
 3 | ▫ | ▫ | ▫ | ▫ | ▫ | ▫ | 
 4 | ▫ 

TypeError: ignored